In [1]:
import numpy as np
import cv2
import apriltag
import solvePos
import basic

sh: 0: cannot open ./stop_sys_ttyPS0.sh: No such file


In [2]:
def load_config(r1,rx,ry):
    tag_positions = {}
    tag_centers = {}
    sftx = np.array([rx, 0, 0])
    sfty = np.array([0, ry, 0])
    tag_positions['3'] = np.array([ [r1/2, -r1/2, 0],[-r1/2, -r1/2, 0], [-r1/2, r1/2, 0], [r1/2, r1/2, 0],])
    tag_positions['2'] = tag_positions['3'] - sftx/2 -sfty/2
    tag_positions['1'] = tag_positions['3'] + sftx/2 -sfty/2
    tag_positions['0'] = tag_positions['3'] + sftx/2 +sfty/2
    tag_positions['5'] = tag_positions['3'] - sftx/2 +sfty/2

    destination = np.array([0,0])
    return tag_positions, destination
tag_positions, destination = load_config(5.00, 76.5, 74.7)

In [3]:
intrinsic = np.array(([207.3099,0,314.3843],[0,205.0819,239.2206],[0,0,1]),dtype=np.double)

distortion = np.array([0.1912,-0.0889,0,0],dtype=np.double)

#mate 40
#intrinsic = np.array(([6662,0,4053.9],[0,6671.3,3071.4],[0,0,1]),dtype=np.double)

#distortion = np.array([-0.0172,0.2871,0,0],dtype=np.double)

In [4]:
options = apriltag.DetectorOptions(families="tag36h11")
detector = apriltag.Detector(options)
img = cv2.imread("./headimg/f5bed3f2416813dd9f432e19725a6ac.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
tags = detector.detect(gray)

In [5]:
for n, tag in enumerate(tags):
    pos = tag_positions[str(tag.tag_id)]
    corners = np.array(tag.corners)
    pos_, rot_ = solvePos.solvePositionAndRotation(pos,tag.corners,intrinsic,distortion)
    print("发现tag", tag.tag_id)
    print("相机位置：", pos_)
    print("旋转角", rot_)

发现tag 2
相机位置： [[-6.06154227]
 [19.35694575]
 [65.2035724 ]]
旋转角 37.33482658375705
发现tag 3
相机位置： [[-15.36013189]
 [ -2.62533812]
 [ 69.70205332]]
旋转角 36.92681773480714
发现tag 5
相机位置： [[ -4.68065147]
 [-11.26321672]
 [ 69.98038974]]
旋转角 38.181717849857385


In [6]:
#tag_num = 0
'''
while tag_num ==0:
    # 拍照
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    tags = detector.detect(gray)
    tag_num = len(tags)
    if tag_num == 0:
        print("not detect")
        for i in range(5):
            run_action("turn003L")
            wait_req()
    else:
        break

tag = tags[0]
pos = tag_positions[str(tag.tag_id)]
corners = np.array(tag.corners)
realPos, ang = solvePos.solvePositionAndRotation(pos,corners,intrinsic,distortion)

shouldRotate = solvePos.getShouldRotAngle(realPos, ang)
'''
shouldRotate = 1000

while abs(shouldRotate) > 10:
    tag_num = 0
    #拍照
    while tag_num ==0:
    # 拍照
        orig_img_path = './headtmp.jpg'
        # !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2> /dev/null
        !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2
        img = cv2.imread("./headtmp.jpg")
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        tags = detector.detect(gray)
        tag_num = len(tags)
        if tag_num == 0:
            print("not detect")
            for i in range(2):
                run_action("turn010L")
                wait_req()
        else:
            break
        
    tag = tags[0]
    pos = tag_positions[str(tag.tag_id)]
    corners = np.array(tag.corners)
    realPos, ang = solvePos.solvePositionAndRotation(pos,corners,intrinsic,distortion)

    shouldRotate = solvePos.getShouldRotAngle(realPos, ang)
    
    if shouldRotate >= 50:
        #左转
        run_action("turn010L")
        wait_req()
        continue
    if shouldRotate >= 20:
        run_action("turn005L")
        wait_req()
        continue
    if shouldRotate >= 10:
        run_action("turn003L")
        wait_req()
        continue
    if shouldRotate <= -50:
        #左转
        run_action("turn010R")
        wait_req()
        continue
    if shouldRotate <= -20:
        run_action("turn005R")
        wait_req()
        continue
    if shouldRotate <= -10:
        run_action("turn003R")
        wait_req()
        continue

distance = (realPos[0]**2+realPos[1]**2)**0.5
run_action("fastForward06")
wait_req()



In [11]:
x = 3**0.5
y = 1
np.degrees(np.arctan2(x,-y))

120.00000000000001